In [12]:
# -*- coding: utf-8 -*-
import os
import math
import pandas as pd
import numpy as np
os.chdir('c:/coding/228')

# estimate 1 : using SDR to estimate 228 death count
(SDR means "Sexualized Death Ratio")

In [13]:
data=pd.read_csv('data.txt',sep='\t')
data.loc[:,'SDR']=data.apply(lambda row:float(row['male'])/row['female'],axis=1)
print data.head(12)

    year  month  male  female       SDR
0   1947      1  5540    4740  1.168776
1   1947      2  4925    4259  1.156375
2   1947      3  5640    4678  1.205643
3   1947      4  5864    4938  1.187525
4   1947      5  5332    4661  1.143961
5   1947      6  4655    4194  1.109919
6   1947      7  4834    4519  1.069706
7   1947      8  4838    4444  1.088659
8   1947      9  4628    4403  1.051102
9   1947     10  4725    4452  1.061321
10  1947     11  4535    4064  1.115896
11  1947     12  4879    4449  1.096651


In [14]:
normalData=data.loc[(data['year']!=1947) & (data['month'].isin([3,4,5])) ,:]
meanSDR=normalData.groupby('month').apply(lambda gData:gData['SDR'].mean())
crossYearSDR=normalData.groupby('month').apply(lambda g:float(g['male'].sum())/g['female'].sum())
crossYearSDR_indexed=pd.DataFrame({u'1949~1951年台灣地區死亡性別比例(SDR)':crossYearSDR})
print 'cross year SDR'
print crossYearSDR_indexed

cross year SDR
       1949~1951年台灣地區死亡性別比例(SDR)
month                                      
3                                  1.099169
4                                  1.100317
5                                  1.048774


In [15]:
def estimate(data,SDR,estimateName='228Estimate'):
    data.loc[:,'SDR']=SDR.tolist()
    data.loc[:,'maleEstimate']=data.apply(lambda row:row['female']*row['SDR'],axis=1)
    data.loc[:,estimateName]=data.apply(lambda row:row['male']-row['maleEstimate'],axis=1)
    estimate=data[estimateName].sum()
    return data,estimate

inciData=data.loc[(data['year']==1947) & (data['month'].isin([3,4,5])) ,:]
                  
data1_1, estimate1_1 = estimate(inciData, crossYearSDR)
print data1_1
print 'estimate1_1, which presented in the 170225 conference : {}\n'.format(math.ceil(estimate1_1))

data1_2, estimate1_2 = estimate(inciData, meanSDR)
print data1_2
print 'estimate1_2, which used meanSDR: {}\n'.format(math.ceil(estimate1_2))
print '(We changed to use meanSDR becuase we thought it might be more reasonable.)'

   year  month  male  female       SDR  maleEstimate  228Estimate
2  1947      3  5640    4678  1.099169   5141.910363   498.089637
3  1947      4  5864    4938  1.100317   5433.363365   430.636635
4  1947      5  5332    4661  1.048774   4888.336619   443.663381
estimate1_1, which presented in the 170225 conference : 1373.0

   year  month  male  female       SDR  maleEstimate  228Estimate
2  1947      3  5640    4678  1.100507   5148.173311   491.826689
3  1947      4  5864    4938  1.100636   5434.939615   429.060385
4  1947      5  5332    4661  1.050343   4895.651019   436.348981
estimate1_2, which used meanSDR: 1358.0

(We changed to use meanSDR becuase we thought it might be more reasonable.)


# estimate 2: using death date in compensation data to modify estimate 1

In [16]:
execfile('cleanCompenData.py')
compen=pd.read_csv('cleanCompenData.txt',sep='\t')
compen['deathDate']=compen['deathDate'].fillna('')
compen['deathDate']=compen['deathDate'].astype('str')
compen['monthSince1947']=compen['deathDate'].apply(lambda x:int(x[4:6])+(int(x[:4])-1947)*12 if x!='' else None)
print compen.loc[:,['name','deathDate','monthSince1947']].head(20)

      name   deathDate  monthSince1947
0   尤萬居  19470306.0             3.0
1   方振東  19470304.0             3.0
2   王天炳  19470309.0             3.0
3   王天富  19470321.0             3.0
4     王木  19470315.0             3.0
5   王平水  19470306.0             3.0
6   王名朝  19470405.0             4.0
7   王有福  19470417.0             4.0
8   王育霖  19470314.0             3.0
9   王來湖                         NaN
10  王東朝  19470301.0             3.0
11  王金火  19470308.0             3.0
12  王金星  19470314.0             3.0
13  王金振                         NaN
14  王金塗  19470301.0             3.0
15  王金樹                         NaN
16  王金興  19470308.0             3.0
17  王垢復  19470307.0             3.0
18  王炳煌  19470303.0             3.0
19  王國家  19470228.0             2.0


In [17]:
monDeath=compen['monthSince1947'].value_counts(dropna=False)

def combineMonth(x):
    if x==2:
        return u'1947年二月'
    elif x>2 and x<6:
        return u'1947年三到五月'
    elif np.isnan(x) or x is None:
        return u'無死亡時間'
    else:
        return u'1947年六月以後'
        
monDeath=monDeath.groupby(combineMonth).sum()
monDeath=monDeath.reindex_axis([u'1947年二月',u'1947年三到五月',u'1947年六月以後',u'無死亡時間'])
monDeath_indexed=pd.DataFrame({u'受補償受難者死亡月份':monDeath})
print monDeath_indexed

                受補償受難者死亡月份
1947年二月                        19
1947年三到五月                   556
1947年六月以後                     8
無死亡時間                       205


In [18]:
monDeath=monDeath.drop(u'無死亡時間')
monDeath=pd.DataFrame({u'死亡人數':monDeath,u'所佔比例':monDeath/monDeath.sum()})
print monDeath

                所佔比例  死亡人數
1947年二月      0.032590        19
1947年三到五月  0.953688       556
1947年六月以後  0.013722         8


In [19]:
estimate2_1= estimate1_1/monDeath.loc[u'1947年三到五月',u'所佔比例']
print 'estimate2_1, which modified estimate1_1(presented in conferenece) : {}'.format(math.ceil(estimate2_1))

estimate2_2= estimate1_2/monDeath.loc[u'1947年三到五月',u'所佔比例']
print 'estimate2_2, which modified estimate1_2: {}'.format(math.ceil(estimate2_2))


estimate2_1, which modified estimate1_1(presented in conferenece) : 1440.0
estimate2_2, which modified estimate1_2: 1424.0


# appendix: Estimating the error margin of 228 daeth count estimation ,using death count of 1947 Jan, Feb, Jun and after.

In [20]:
testData=data.loc[(data['year']==1947) & (~data['month'].isin([3,4,5])) ,:]
testNormalData=data.loc[(data['year']!=1947) & (~data['month'].isin([3,4,5])) ,:]

testMeanSDR=testNormalData.groupby('month').apply(lambda gData:gData['SDR'].mean())
testCrossYearSDR=testNormalData.groupby('month').apply(lambda g:float(g['male'].sum())/g['female'].sum())

testData1_1, errorEstimate1_1=estimate(testData, testMeanSDR, estimateName='errorEstimate')
print testData1_1
print '\naverage estimation error of male normal death, using cross year SDR:{}\n'.format(
        -1*errorEstimate1_1/testData['male'].sum())

testData1_2, errorEstimate1_2=estimate(testData,testCrossYearSDR,estimateName='errorEstimate')
print testData1_2
print '\naverage estimation error of male normal death, using meanSDR:{}\n'.format(
        -1*errorEstimate1_2/testData['male'].sum())              



    year  month  male  female       SDR  maleEstimate  errorEstimate
0   1947      1  5540    4740  1.109664   5259.808206     280.191794
1   1947      2  4925    4259  1.100968   4689.022307     235.977693
5   1947      6  4655    4194  1.080071   4529.816523     125.183477
6   1947      7  4834    4519  1.051487   4751.671874      82.328126
7   1947      8  4838    4444  1.049304   4663.104957     174.895043
8   1947      9  4628    4403  1.052443   4633.905851      -5.905851
9   1947     10  4725    4452  1.096763   4882.788117    -157.788117
10  1947     11  4535    4064  1.087600   4420.008018     114.991982
11  1947     12  4879    4449  1.074337   4779.723954      99.276046

average estimation error of male normal death, using cross year SDR:-0.0217899904071

    year  month  male  female       SDR  maleEstimate  errorEstimate
0   1947      1  5540    4740  1.109323   5258.191925     281.808075
1   1947      2  4925    4259  1.101768   4692.427959     232.572041
5   1947      6 